In [6]:

from decimal import getcontext
from decimal import Decimal


class SourceStates: #контейнер, содержащий алфавит сообщения для кодирования (состояния источника и вероятности их появления)
    @staticmethod
    def create_dict(message: str) -> dict: #статический метод для создания словаря [символ, кол-во появлений]
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states
    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict: #статический метод для создания словаря [символ, вероятность появления символа]
        freq_dict = {}
        for char, count in states.items():
            freq_dict[char] = count / len(message)
        return freq_dict
    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict: #левые границы кодирования (накопительные вероятности)
        left_borders_dict = {}
        sum_l = 0.0
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum_l
            sum_l += freq 
        return left_borders_dict #т.е получаем для каждого символа начало его отрезка внутри [0,1)
    def __init__(self, message):
        self.states = self.create_dict(message) #сколько раз каждый символ встречается в сообщении
        self.freq_states = self.create_freq_dict(self.states, message) #вероятность появления каждого символа в сообщении
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states) 

getcontext().prec = 500 #установка точности (кол-во знаков после запятой)

class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Decimal(0.0) #левая граница отрезка кодирования c выставленной точностью
        self.right = Decimal(1.0) #правая граница отрезка кодирования c выставленной точностью
        self.source_states = SourceStates(self.message) #из объекта этого класса получаем все данные о состояниях источника (см. выше)
        
    def encode(self) -> float:
        for char in self.message:
            left_border = Decimal(self.source_states.left_borders_dict[char]) #левая граница текущего символа
            right_border = Decimal(self.source_states.left_borders_dict[char] + self.source_states.freq_states[char]) #правая граница текущего символа
            current_range = self.right - self.left #длина текущего отрезка
            self.right = self.left + current_range * right_border #усекаем отрезок в соответствии с правой границей текущего символа
            self.left = self.left + current_range * left_border #усекаем отрезок в соответствии с левой границей текущего символа
        
        return (self.left + self.right) / 2 #возвращаем середину результирующего отрезка
class ArithmeticDecode:
    def __init__(self, coded_val: float, source_states: SourceStates, stop_symbol: str):
        self.coded_val = Decimal(coded_val) #закодированное слово с выставленной точностью
        self.stop_symbol = stop_symbol
        self.source_states = source_states #из объекта этого класса получаем все данные о состояниях источника (см. выше)
        self.right = Decimal(1.0) #левая граница отрезка кодирования c выставленной точностью
        self.left = Decimal(0.0) #правая граница отрезка кодирования c выставленной точностью
        
    def decode(self) -> str:
        if self.stop_symbol not in self.source_states.freq_states.keys():
            print("Stop symbol not in alphabet")
            return ""
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        found_stop_symbol = False
        while not found_stop_symbol:
            current_range = self.right - self.left #длина текущего отрезка
            if current_range == 0: #выходим из цикла на границе точности чтобы избежать деления на ноль
                break
            scaled_value = (self.coded_val - self.left) / current_range #приводим закодированное значение к интервалу [0,1)
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char] #правая граница
                if add_left_freq <= scaled_value < add_right_freq: #попадает ли приведенное значение в отрезок текущего символа: если да, то добавляем и обновляем границы, если нет, то идём на следующую итерацию цикла for each
                    if char == self.stop_symbol: #выходим из цикла если нашли символ останова
                        found_stop_symbol = True
                        break
                    decoded_message += char
                    self.right = self.left + current_range * Decimal(add_right_freq) #обновляем границы
                    self.left = self.left + current_range * Decimal(add_left_freq)
        return decoded_message

file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
data = file.read()
file.close()

coder = ArithmeticEncode(data)
decoder = ArithmeticDecode(coder.encode(), SourceStates(data), "\n")
print(f"Frequencies: {coder.source_states.freq_states}")
print(f"Encoded: {coder.encode()}")
print(f"Decoded: {decoder.decode()}")


Frequencies: {'В': 0.004739336492890996, 'с': 0.04739336492890995, 'е': 0.04739336492890995, ' ': 0.13744075829383887, 'и': 0.037914691943127965, 'м': 0.04265402843601896, 'в': 0.052132701421800945, 'о': 0.11374407582938388, 'л': 0.023696682464454975, 'ы': 0.023696682464454975, 'т': 0.05687203791469194, 'а': 0.04265402843601896, 'к': 0.023696682464454975, 'г': 0.004739336492890996, 'ф': 0.004739336492890996, 'п': 0.009478672985781991, 'р': 0.03317535545023697, 'н': 0.037914691943127965, 'у': 0.018957345971563982, '0': 0.04265402843601896, 'д': 0.04265402843601896, '2': 0.004739336492890996, '5': 0.009478672985781991, ',': 0.004739336492890996, 'ж': 0.004739336492890996, '8': 0.004739336492890996, '-': 0.004739336492890996, 'з': 0.004739336492890996, 'я': 0.02843601895734597, 'й': 0.014218009478672985, 'ч': 0.014218009478672985, '1': 0.037914691943127965, '.': 0.009478672985781991, 'Э': 0.004739336492890996, '\n': 0.004739336492890996}
Encoded: 0.0000352545963983416040845454364497810026

In [9]:
from fractions import Fraction
from decimal import Decimal

class SourceStates:  # контейнер, содержащий алфавит сообщения для кодирования (состояния источника и вероятности их появления)
    @staticmethod
    def create_dict(message: str) -> dict:  # статический метод для создания словаря [символ, кол-во появлений]
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states

    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict:  # статический метод для создания словаря [символ, вероятность появления символа]
        freq_dict = {}
        total = len(message)
        for char, count in states.items():
            freq_dict[char] = Fraction(count, total)  #используем дроби для вероятностей
        return freq_dict

    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict:  # левые границы кодирования (накопительные вероятности)
        left_borders_dict = {}
        sum_fraction = Fraction(0, 1)  #инициализация дроби 0/1
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum_fraction
            sum_fraction += freq
        return left_borders_dict

    def __init__(self, message):
        self.states = self.create_dict(message)
        self.freq_states = self.create_freq_dict(self.states, message)
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states)

#КОММЕНТАРИИ К АЛГОРИТМАМ КОДИРОВАНИЯ И ДЕКОДИРОВАНИЯ В ЛИСТИНГЕ ДЛЯ ДЕСЯТИЧНЫХ ДРОБЕЙ
class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Fraction(0, 1)  #левая граница 0/1 = 0
        self.right = Fraction(1, 1)  #правая граница 1/1 = 1
        self.source_states = SourceStates(self.message)

    def encode(self) -> Fraction:
        for char in self.message:
            left_border = self.source_states.left_borders_dict[char]
            right_border = left_border + self.source_states.freq_states[char]
            current_range = self.right - self.left
            self.right = self.left + current_range * right_border
            self.left = self.left + current_range * left_border

        return (self.left + self.right) / 2  #возвращает результат дробью

class ArithmeticDecode:
    def __init__(self, coded_val: Fraction, source_states: SourceStates, stop_symbol: str):
        self.stop_symbol = stop_symbol
        self.coded_val = coded_val
        self.source_states = source_states
        self.right = Fraction(1, 1)  #левая граница 0/1 = 0
        self.left = Fraction(0, 1)  #правая граница 1/1 = 1

    def decode(self) -> str:
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        found_stop_symbol = False
        while not found_stop_symbol:
            current_range = self.right - self.left
            if current_range == 0: 
                break
            scaled_value = (self.coded_val - self.left) / current_range
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char]
                if add_left_freq <= scaled_value < add_right_freq:
                    if char == self.stop_symbol: 
                        found_stop_symbol = True
                        break
                    decoded_message += char
                    self.right = self.left + current_range * add_right_freq
                    self.left = self.left + current_range * add_left_freq
        return decoded_message

file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
data = file.read()
file.close()

code = ArithmeticEncode(data)
encoded_value = code.encode()
decode = ArithmeticDecode(encoded_value, SourceStates(data), "\n")
print(f"Frequencies: {code.source_states.freq_states}")
print(f"Encoded: {encoded_value}")
print(f"Decoded: {decode.decode()}")

Frequencies: {'В': Fraction(1, 211), 'с': Fraction(10, 211), 'е': Fraction(10, 211), ' ': Fraction(29, 211), 'и': Fraction(8, 211), 'м': Fraction(9, 211), 'в': Fraction(11, 211), 'о': Fraction(24, 211), 'л': Fraction(5, 211), 'ы': Fraction(5, 211), 'т': Fraction(12, 211), 'а': Fraction(9, 211), 'к': Fraction(5, 211), 'г': Fraction(1, 211), 'ф': Fraction(1, 211), 'п': Fraction(2, 211), 'р': Fraction(7, 211), 'н': Fraction(8, 211), 'у': Fraction(4, 211), '0': Fraction(9, 211), 'д': Fraction(9, 211), '2': Fraction(1, 211), '5': Fraction(2, 211), ',': Fraction(1, 211), 'ж': Fraction(1, 211), '8': Fraction(1, 211), '-': Fraction(1, 211), 'з': Fraction(1, 211), 'я': Fraction(6, 211), 'й': Fraction(3, 211), 'ч': Fraction(3, 211), '1': Fraction(8, 211), '.': Fraction(2, 211), 'Э': Fraction(1, 211), '\n': Fraction(1, 211)}
Encoded: 935004745239780085144930204032589417987470086611540638932414005562120019742061861921194636442739010163469280070224135456268030187518726653981368559807383745062998271

In [30]:
file = open(r"..\cache\dz\ex1.txt", "r", encoding="utf-8")
print(file.read())
file.close()

Все символы такого алфавита пронумерованы от 0 до 255, а каждому номеру соответствует 8-разрядный двоичный код от 00000000 до 11111111. Этот код является порядковым номером символа в двоичной системе счисления.
